## vLLM

vLLM 是一个高吞吐量且内存高效的推理和服务引擎，专为大型语言模型（LLM）设计。它通过利用先进的内存管理技术，如 PagedAttention，有效管理注意力键和值的内存，从而优化 LLM 的服务和执行。这使得能够对传入请求进行连续批处理并实现快速模型执行，使 vLLM 成为大规模部署和服务 LLM 的强大工具。

vLLM 支持与流行的 Hugging Face 模型无缝集成，并提供多种解码算法，包括并行采样和束搜索。它还支持张量并行和流水线并行以实现分布式推理，使其成为一个灵活且易于使用的 LLM 推理解决方案（详见完整文档）。

## 前置条件

配置当前订阅，默认的工作区和资源组，如下图所示，从右上角菜单查看

- 工作区: 工作区是与同事协作创建机器学习工件和分组相关工作的场所。例如，实验、作业、数据集、模型、组件和推理端点。
- 资源组: Azure 中的资源组是一个逻辑容器, 它允许您根据资源的生命周期和安全需求将这些资源作为一个整体进行管理。资源组中的资源可以包括虚拟机、存储账户和虚拟网络等。

![描述文本](./setup.png)

In [6]:
resource_group="test-ml-rg2"
subscription="b1d92895-527c-4a67-91d2-8f653d9ee248"
workspace="test-ml-ws"

执行配置命令

In [12]:
!az account set --subscription {subscription}
!az configure --defaults workspace={workspace} group={resource_group}

## 第一步，在 AzureML 上为 vLLM 创建自定义环境

环境（Environment）是 Azure 机器学习中的一个概念，它定义了用于训练和推理的计算资源。环境可以包含多个组件，例如 Python 包、依赖项和配置。

这里通过 `environment.yml` 文件指定环境配置：更复杂的配置参考 [CLI (v2) environment YAML schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-environment?view=azureml-api-2)

```yaml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: vllm
build:
  path: .
  dockerfile_path: Dockerfile
```


1. $schema: …environment.schema.json
- 指定了这个 YAML 文件所遵循的 JSON Schema 地址。
- AzureML 在解析时会根据这个 schema 对文件内容做校验，确保字段和格式都符合最新的环境定义规范。
2. name: vllm
- 定义了在 AzureML 中注册的环境名称，这里你把它命名为 vllm。
- 后续在创建计算集群或推理部署时，就可以通过这个名称来引用刚刚定义好的环境。
3. build:
这个部分告诉 AzureML 如何构建底层的 Docker 镜像。
- path: .
  - 构建上下文（build context）的目录路径。
  - . 表示当前文件所在目录，所有的依赖文件（如 Dockerfile、代码、依赖列表等）都应该放在这里或其子目录中。
- dockerfile_path: Dockerfile
  - 指定要使用的 Dockerfile 文件名或路径。

执行命令构建环境

In [ ]:
!az ml environment create -f environment.yml

## 第二步，创建 AzureML 托管的在线端点

接下来开始正式部署, 首先创建托管在线端点

托管在线端点（Managed Online Endpoint）是 Azure 机器学习中的一个概念，它允许您在 Azure 机器学习平台上部署和管理在线推理服务。托管在线端点提供了一种简化的方式来部署和管理模型，并支持自动缩放和监控。

![](./online-endpoint.png)

创建 `endpoint.yml` 文件，更多配置参考 [CLI (v2) managed online endpoint YAML schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-endpoint-online?view=azureml-api-2)

```yml
$schema: https://azuremlsdk2.blob.core.windows.net/latest/managedOnlineEndpoint.schema.json
name: vllm-hf
auth_mode: key
```

1. $schema: …managedOnlineEndpoint.schema.json
  - 指定了该 YAML 文件所遵循的 JSON Schema。
  - Azure ML 在解析此文件时，会根据这个 Schema 验证字段名称、类型和结构是否正确。
  - 这里指向的是最新版本的 “托管在线端点” 定义，确保你使用的是最新的字段和语法规范。
2. name: vllm-hf
  - 为这个托管的在线端点（Endpoint）设置一个唯一名称，便于在同一个 Azure ML 工作区中进行管理和调用。
  - 该名称会用于 CLI、SDK 或 REST API 中引用该端点，比如后续创建部署或发送推理请求时用到。
3. auth_mode: key
  - 指定该端点的认证方式为 “Key” 模式。
  - 意味着访问这个端点时，需要在请求头中带上由 Azure ML 生成的访问密钥（Primary/Secondary Key）。
  - 其他可选模式还包括 aad_token, aml_token 等，但这里使用最简单的密钥认证。

执行命令创建部署

In [ ]:
!az ml online-endpoint create -f endpoint.yml

接下来的步骤我们需要查看一下刚刚创建的 Docker 镜像地址
可以从 AzureML Studio -> Environments -> vlllm: 看到

![](./arc.png)

最后，我们创建 `deployment.yml` 文件配置部署相关设置，并且部署模型

```yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: current
endpoint_name: vllm-hf
environment_variables:
  MODEL_NAME: meta-llama/Llama-3.1-8B-Instruct # define the model name using the identifier from HG
  VLLM_ARGS: "--enable-auto-tool-choice --tool-call-parser llama3_json"
  HUGGING_FACE_HUB_TOKEN: xxxxxxxxxxxxxx # Add your HF API key here
environment:
  image: xxxxxxxxx.azurecr.io/azureml/azureml_xxxxxxxxxxx # Replace with your own image
  inference_config:
    liveness_route:
      port: 8000
      path: /ping 
    readiness_route:
      port: 8000
      path: /health
    scoring_route:
      port: 8000
      path: /
instance_type: Standard_NC24ads_A100_v4
instance_count: 1
request_settings:
    request_timeout_ms: 60000
    max_concurrent_requests_per_instance: 16 
liveness_probe:
  initial_delay: 10
  period: 10
  timeout: 2
  success_threshold: 1
  failure_threshold: 30
readiness_probe:
  initial_delay: 120
  period: 10
  timeout: 2
  success_threshold: 1
  failure_threshold: 30
```

由于 vLLM 不支持分别用于就绪性和存活性的探针，我们需要确保模型完全加载后再触发第一个探针。这就是为什么我们将 readiness_probe.initial_delay 增加到 120 秒。

对于更大的模型，我们还应按照 vLLM 的文档使用张量并行推理（模型在单个节点上但跨多个 GPU）的方法，通过向 VLLM_ARGS 添加--tensor-parallel-size <GPU 数量>。由于我们示例中使用的是单个 A100 GPU（Standard_NC24ads_A100_v4），因此这不是必需的。

request_settings 在一定程度上取决于我们的实例类型/大小，可能需要一些手动调优以使模型运行得正确且高效。目标是在并发量（max_concurrent_requests_per_instance）和排队时间之间找到一个良好的平衡，以避免端点侧触发 request_timeout_ms，或客户端侧出现任何 HTTP 超时。

这两种情况都会导致 HTTP 429，客户端需要实现指数退避（例如通过 tenacity 库）。

最后，我们可以执行命令部署模型：

In [31]:
!az ml online-deployment create -f deployment.yml --all-traffic

(UserError) A deployment with this name already exists. If you are trying to create a new deployment, use a
different name. If you are trying to update an existing deployment, use `az ml online-deployment update` instead.


如果已经创建，执行更新操作

In [33]:
!az ml online-deployment update -f deployment.yml

Check: endpoint vllm-hf exists
(ResourceNotReady) User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready Running .. - Running .. / Running .. \ Running .. \ Running .. / Running .. | Running .. \ Running .. - Running .. / Running .. | Running .. - Running .. / Running .. \ Running .. - Running .. / Running .. | Running .. \ Running .. - Interrupted ..
Code: ResourceNotReady
Message: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready


如果提示 "Specified deployment [current] failed during initial provisioning and is in an unrecoverable state. Delete and re-create" 
执行删除操作

In [28]:
!az ml online-deployment delete -n current --endpoint-name vllm-hf --yes

 / Finished ..

## 第 3 步：测试部署

首先我们获取端点的评分 URI 和 API 密钥:

In [ ]:
!az ml online-endpoint show -n vllm-hf
!az ml online-endpoint get-credentials -n vllm-hf

对于补全模型，我们可以使用以下 Python 代码片段调用该端点：

In [ ]:
import requests

url = "https://vllm-hf.polandcentral.inference.ml.azure.com/v1/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer xxxxxxxxxxxx"
}
data = {
    "model": "openai-community/gpt2",
    "prompt": "San Francisco is a",
    "max_tokens": 200,
    "temperature": 0.7
}

response = requests.post(url, headers=headers, json=data)
print(response.json())

## 结论

总之，在 Azure 机器学习托管在线端点上使用 vLLM 部署大型语言模型是一种简单有效的方式，可以实现实时的 AI 驱动应用。通过遵循所分享的步骤——从环境设置到部署测试——您可以快速将 Llama-3.1-8B-Instruct 等先进模型集成到工作流程中。借助 vLLM 的优化性能和对函数调用的支持，您的应用能够无缝处理复杂任务并与其他系统交互。此配置帮助您构建更智能、更快速且更具可扩展性的 AI 解决方案。

## Troubleshoot

### 解决存储账户网络访问权限问题

配置允许公网访问

In [ ]:
# Enable public network access for Azure storage account using az cli
!az storage account update \
    --name $(az storage account list -g {resource_group} --query "[0].name" -o tsv) \
    --resource-group {resource_group} \
    --public-network-access Enabled \
    --default-action Allow

配置允许存储账户密钥访问

In [ ]:
# Enable storage account key access using az cli
!az storage account update \
    --name $(az storage account list -g {resource_group} --query "[0].name" -o tsv) \
    --resource-group {resource_group} \
    --allow-shared-key-access true